In [ ]:
!pip install kaggle


In [ ]:
pip install gradio


In [ ]:
import gradio as gr

In [ ]:
from google.colab import files
files.upload()


In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
#casting envi

In [ ]:
!kaggle datasets download -d aryashah2k/indian-medicinal-leaves-dataset


In [ ]:
!unzip /content/indian-medicinal-leaves-dataset.zip


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
import cv2
import os
import numpy as np

In [ ]:
dataset_path= '/content/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset'

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
def load_and_preprocess_images(dataset_path, image_size=(128, 128)):
    images = []
    labels = []
#0=0 white 1=255 black 0.5=128 grey
    # Iterate through class directories
    for class_dir in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_dir)
        class_label = int(class_dir)  # Convert directory name to label if needed

        # Iterate through images in the class directory
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)

            # Read and preprocess the image
            image = cv2.imread(image_path)
            image = cv2.resize(image, image_size)
            image = image.astype(np.float32) / 255.0  # Normalize pixel values

            # Append the image and label to lists
            images.append(image)
            labels.append(class_label)

    return np.array(images), np.array(labels)


# Constants
IMAGE_RES = 224
BATCH_SIZE = 32
# Load the dataset
data = tf.keras.utils.image_dataset_from_directory(
    '/content/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset',
    image_size=(IMAGE_RES, IMAGE_RES),
    batch_size=BATCH_SIZE,
    validation_split=0.2,  # 20% of the data will be used for testing
    subset="training",     # Specify "training" to get the training subset
    seed=42,
    label_mode='int'
)
num_examples = data.cardinality().numpy()
num_classes = len(data.class_names)

# Split the data into training and testing
testing_data = tf.keras.utils.image_dataset_from_directory(
    '/content/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset',
    image_size=(IMAGE_RES, IMAGE_RES),
    batch_size=BATCH_SIZE,
    validation_split=0.2,  # No validation split for testing
    seed=42,
    label_mode='int',
    subset="validation"    # Specify "validation" to get the testing subset
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

In [ ]:
IMAGE_RES = 224  # InceptionV3 input shape
BATCH_SIZE = 32
def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES)) / 255.0
    return image, label
train_data = data
# Define the validation split ratio
validation_split = 0.2
num_validation_samples = int(num_examples * validation_split)
validation_data = data.take(num_validation_samples)

train_batches = train_data.map(format_image).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_batches = validation_data.map(format_image).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Create the base model (MobileNetV2)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMAGE_RES, IMAGE_RES, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model
base_model.trainable = False

# Add custom layers for classification
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

# Create the final model
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(train_batches, validation_data=validation_batches, epochs=10)

# Evaluate the model on the testing dataset
testing_batches = testing_data.map(format_image).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_loss, test_accuracy = model.evaluate(testing_batches)
print("Test Accuracy:", test_accuracy)
print("Test Loss:", test_loss)

In [ ]:
# Define the path where you want to save the model
model_path = '/content/modelsaved/'

# Save the model
tf.keras.models.save_model(model, model_path)

# Print the path where the model is saved
print("Model saved at:", model_path)


In [ ]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np

# Load the pre-trained model
model = tf.keras.models.load_model("/content/modelsaved")  # Replace with the actual path to your model file
IMAGE_RES = 224  # Assuming this is the image resolution used during training

# Create a dictionary to map class indices to class names
class_names = ["Aloevera", "Amla", "Amruthaballi", "Arali", "Astma_weed", "Badipala", "Balloon_Vine", "Bamboo",
               "Beans", "Betel", "Bhrami", "Bringaraja", "Caricature", "Castor", "Catharanthus", "Chakte",
               "Chilly", "Citron lime (herelikai)", "Coffee", "Common rue(naagdalli)", "Coriender", "Curry",
               "Doddpathre", "Drumstick", "Ekka", "Eucalyptus", "Ganigale", "Ganike", "Gasagase", "Ginger",
               "Globe Amarnath", "Guava", "Henna", "Hibiscus", "Honge","Insulin", "Jackfruit", "Jasmine", "Kambajala",
               "Kasambruga", "Kohlrabi", "Lantana", "Lemon", "Lemongrass", "Malabar_Nut", "Malabar_Spinach", "Mango",
               "Marigold", "Mint", "Neem", "Nelavembu", "Nerale", "Nooni", "Onion", "Padri", "Palak(Spinach)", "Papaya",
               "Parijatha", "Pea", "Pepper", "Pomoegranate", "Pumpkin", "Raddish", "Rose", "Sampige", "Sapota",
               "Seethaashoka", "Seethapala", "Spinach1", "Tamarind", "Taro", "Tecoma", "Thumbe", "Tomato", "Tulsi",
               "Turmeric", "ashoka", "camphor", "kamakasturi", "kepala"]  # Replace with your actual class names

# Load the uses of the plants from a text file
def load_plant_uses(file_path):
    plant_uses = {}
    with open(file_path, 'r') as file:
        for line in file:
            name, uses = line.strip().split(',', 1)
            plant_uses[name] = uses
    return plant_uses

# Load the plant uses into a dictionary
plant_uses = load_plant_uses('/content/plant_uses.txt')  # Replace with the actual path to your text file

def preprocess_image(input_image):
    # Load the image and convert it to an array
    input_image_array = img_to_array(input_image)

    # Resize the image to match the model's input shape
    input_image_array = tf.image.resize(input_image_array, (IMAGE_RES, IMAGE_RES))

    # Normalize the image
    input_image_array = input_image_array / 255.0

    # Add batch dimension
    input_image_array = np.expand_dims(input_image_array, axis=0)

    return input_image_array

def classify_image(input_image):
    try:
        # Preprocess the input image
        input_image_array = preprocess_image(input_image)

        # Make predictions
        predictions = model.predict(input_image_array)
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        predicted_class_name = class_names[predicted_class_index]

        # Get the uses of the predicted plant
        uses = plant_uses.get(predicted_class_name, "No uses found")

        return f"{predicted_class_name}: {uses}"
    except Exception as e:
        return str(e)  # Return the error message as output

# Define the Gradio interface
gr.Interface(fn=classify_image, inputs="image", outputs="text", title="Medicinal Plant Detection").launch()


In [ ]:
pip install telepot

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import telepot
from telepot.loop import MessageLoop
from io import BytesIO
import requests

# Load the pre-trained model
model = tf.keras.models.load_model("/content/modelsaved")  # Replace with the actual path to your model file
IMAGE_RES = 224  # Assuming this is the image resolution used during training

# Create a dictionary to map class indices to class names
class_names = ["Aloevera", "Amla", "Amruthaballi", "Arali", "Astma_weed", "Badipala", "Balloon_Vine", "Bamboo",
               "Beans", "Betel", "Bhrami", "Bringaraja", "Caricature", "Castor", "Catharanthus", "Chakte",
               "Chilly", "Citron lime (herelikai)", "Coffee", "Common rue(naagdalli)", "Coriender", "Curry",
               "Doddpathre", "Drumstick", "Ekka", "Eucalyptus", "Ganigale", "Ganike", "Gasagase", "Ginger",
               "Globe Amarnath", "Guava", "Henna", "Hibiscus", "Honge","Insulin", "Jackfruit", "Jasmine", "Kambajala",
               "Kasambruga", "Kohlrabi", "Lantana", "Lemon", "Lemongrass", "Malabar_Nut", "Malabar_Spinach", "Mango",
               "Marigold", "Mint", "Neem", "Nelavembu", "Nerale", "Nooni", "Onion", "Padri", "Palak(Spinach)", "Papaya",
               "Parijatha", "Pea", "Pepper", "Pomoegranate", "Pumpkin", "Raddish", "Rose", "Sampige", "Sapota",
               "Seethaashoka", "Seethapala", "Spinach1", "Tamarind", "Taro", "Tecoma", "Thumbe", "Tomato", "Tulsi",
               "Turmeric", "ashoka", "camphor", "kamakasturi", "kepala"]  # Replace with your actual class names

# Load the uses of the plants from a text file
def load_plant_uses(file_path):
    plant_uses = {}
    with open(file_path, 'r') as file:
        for line in file:
            name, uses = line.strip().split(',', 1)
            plant_uses[name] = uses
    return plant_uses

# Load the plant uses into a dictionary
plant_uses = load_plant_uses('/content/plant_uses.txt')  # Replace with the actual path to your text file

def preprocess_image(input_image):
    # Convert the image to an array
    input_image_array = img_to_array(input_image)

    # Resize the image to match the model's input shape
    input_image_array = tf.image.resize(input_image_array, (IMAGE_RES, IMAGE_RES))

    # Normalize the image
    input_image_array = input_image_array / 255.0

    # Add batch dimension
    input_image_array = np.expand_dims(input_image_array, axis=0)

    return input_image_array

def classify_image(input_image):
    try:
        # Preprocess the input image
        input_image_array = preprocess_image(input_image)

        # Make predictions
        predictions = model.predict(input_image_array)
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        predicted_class_name = class_names[predicted_class_index]

        # Get the uses of the predicted plant
        uses = plant_uses.get(predicted_class_name, "No uses found")

        return f"{predicted_class_name}: {uses}"
    except Exception as e:
        return str(e)  # Return the error message as output

# Define the function to handle incoming Telegram messages
def handle(msg):
    content_type, chat_type, chat_id = telepot.glance(msg)

    if content_type == 'photo':
        # Get the largest photo size
        photo = msg['photo'][-1]
        file_id = photo['file_id']

        # Get the file path
        file_info = bot.getFile(file_id)
        file_path = file_info['file_path']

        # Download the image
        image_url = f"https://api.telegram.org/file/bot{TOKEN}/{file_path}"
        response = requests.get(image_url)
        image = load_img(BytesIO(response.content), target_size=(IMAGE_RES, IMAGE_RES))

        # Classify the image
        result = classify_image(image)

        # Send the result back to the user
        bot.sendMessage(chat_id, result)
    else:
        bot.sendMessage(chat_id, "Please send a photo of a plant for classification.")

# Replace 'YOUR_BOT_TOKEN' with your actual bot token
TOKEN = '6879192190:AAGIBMNBoql4BNa-u7qMqtA7VAMfrwUwKiI'
bot = telepot.Bot(TOKEN)

# Start listening to Telegram messages
MessageLoop(bot, handle).run_as_thread()

print('Listening for incoming messages...')

# Keep the program running
import time
while True:
    time.sleep(10)
